# Configuration basics

Ablator embraces a versatile configuration system that encapsulates every facet of training machine learning models, covering not only the model architecture but also the training environment itself.

Think of this configuration system as a blueprint for crafting experiments. By leveraging this system, Ablator orchestrates the creation and setup of experiments, seamlessly integrating the necessary configurations. 

Furthermore, Ablator offers the flexibility to dynamically construct a hierarchical configuration through composition. You have the choice to override settings either via `yaml` configuration files and command-line inputs, or to directly use Python objects and classes. Explore [these illustrative examples](./GettingStarted-more-demos.ipynb) or delve into [this section](#alternatives-to-constructing-configuration-objects) of this tutorial to gain insights into implementing these two approaches.

In this tutorial, we will explain all configuration-related concepts in Ablator. We will also demonstrate how to configure an experiment in Ablator.

## Configuration categories

For our framework, configuration is organized in different categories:
- [Model configuration](#model-configuration).
- [Training configuration](#training-configuration).
- [Optimizer and Scheduler configuration](#optimizer-and-scheduler-configurations).
- [Running configurations](#running-configurations) (either for training a single model or training multiple models in parallel).
<div class="alert alert-info">

Note

All configuration classes must inherits from `ConfigBase` and decorated with `@configclass` decorator.

</div>

These configuration classes will be used together to configure an experiment.

### Model Configuration

This configuration, required when creating the run configuration (`RunConfig` or `ParallelConfig`), is used to define hyperparameters specific to the model of interest. Later, you can use this configuration to construct the model.

There are 2 steps that are required after defining a model config class for your model:

- Pass the model config to the main model's constructor so you can construct the model using the attributes that's defined in the config.

- Create a custom [running configuration class](#running-configurations) and update `model_config` argument to be of type model config.

Below is an example of a simple 1-layer neural network model, with configuration for input size (to be inferred); hidden layer dimension, activation function, and dropout rate (all of which are stateful);

```python
from ablator import RunConfig, ModelConfig, Stateless, Derived, configclass

import torch.nn as nn
import torch

class MyModelConfig(ModelConfig):
    inp_size: Derived[int]
    hidden_dim: int
    activation: str
    dropout: float

class MyCustomModel(nn.Module):
    def __init__(self, config: MyModelConfig) -> None:
        super().__init__()
        self.linear = nn.Linear(config.inp_size, config.hidden_dim)
        self.dropout = nn.Dropout(config.dropout)
        if config.activation == "relu":
            self.activate = nn.ReLU()
        elif config.activation == "elu":
            self.activate = nn.ELU()

    def forward(self, x: torch.Tensor):
        out = self.linear(x)
        out = self.dropout(out)
        out = self.activate(out)

        return {"preds": out, "labels": out}, x.sum().abs()

model_config = MyModelConfig(hidden_dim=100, activation="relu", dropout=0.3)

# Define custom run config class:
# @configclass
# class CustomRunConfig ...
```

In addition, you can create a search space over these parameters and then use ablator to run Hyperparameter optimization (HPO). A sample use case for this is when you want to test different values for model size, number of layers, activation functions, etc. You can do this by creating a custom `ModelConfig` class for the model and include these parameters. Refer to the [Hyperparameter Optimization](./HPO-tutorial.ipynb) tutorial for more details.

### Training Configuration

This configuration class, required when creating the run configuration (`RunConfig` or `ParallelConfig`), defines the training setting (e.g., batch size, number of epochs, the optimizer to use, etc.). Two important attributes to metion are `optimizer_config` and `scheduler_config`. As the names suggest, they configure the optimizer and scheduler to be used in the training process.

| Parameter         | Usage                                                                 |
|-------------------|-----------------------------------------------------------------------|
| dataset _(str)_           | dataset name. maybe used in custom dataset loader functions.          |
| batch_size _(int)_        | batch size.                                                           |
| epochs _(int)_            | number of epochs to train.                                            |
| optimizer_config _(OptimizerConfig)_  | optimizer configuration. (check ``OptimizerConfig`` for more details) |
| scheduler_config _(Optional[SchedulerConfig])_  | scheduler configuration. (check ``SchedulerConfig`` for more details) |
| rand_weights_init _(bool, defaults to True)_ | whether to initialize model weights randomly.                         |

### Optimizer and Scheduler Configurations

`OptimizerConfig` is used to configure the optimizer for the training process. Currently, we support `SGD` optimizer, `Adam` optimizer, and `AdamW` optimizer.

`SchedulerConfig`, on the other hand, can be used to configure the learning rate scheduler for the training process. Currently, we support `StepLR` scheduler, `OneCycleLR` scheduler, and `ReduceLROnPlateau` scheduler.

Both of these config classes have similar arguments:
| Parameter | Usage                                                                                                                                                                       |
|-----------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| name _(str)_      | The type of the scheduler or optimizer, this can be any in ``['sgd', 'adam', 'adamw']`` for optimizers and <br> in ``['none', 'step', 'cycle', 'plateau']`` for schedulers.  |
| arguments _(OptimizerArgs)_ | The arguments for the scheduler or optimizer, specific to a certain type of scheduler or scheduler. For opimizer, must include an item for learning rate, e.g. `{"lr": 0.1}`                                                                         |

The table below shows how arguments can be defined for each type of optimzer:

| Optimizer type | Arguments                                                                                                                                                                                                                                                                           |
|----------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| sgd            | `weight_decay` _(defaults to 0.0)_: Weight decay rate<br> `momentum` _(defaults to 0.0)_: Momentum factor<br>                                                                                                                                                                                                           |
| adam           | `betas` _(defaults to (0.5, 0.9))_: Coefficients for computing running averages of gradient and its square.<br/> `weight_decay` _(defaults to 0.0)_: Weight decay rate.<br/>                                                                                                    |
| adamw          | `betas` _(defaults to (0.9, 0.999))_: Coefficients for computing running averages of gradient and its square (default is ``(0.9, 0.999)``).<br/> `eps` _(defaults to 1e-8)_: Term added to the denominator to improve numerical stability (default is ``1e-8``).<br/> `weight_decay` _(defaults to 0.0)_: Weight decay rate (default is ``0.0``).<br/> |

The table below shows how arguments can be defined for each type of scheduler:

| Scheduler type | Arguments                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                |
|----------------|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| cycle          | `max_lr` : Upper learning rate boundaries in the cycle.<br> `total_steps` : The total number of steps to run the scheduler in a cycle.<br> `step_when` _(defaults to `"train"`)_: The step type at which the `scheduler.step()` should be invoked: ``'train'``, ``'val'``, or ``'epoch'``.             |
| plataeu        | `patience` _(defaults to 10)_: Number of epochs with no improvement after which learning rate will be reduced.<br> `min_lr` _(defaults to 1e-5)_: A lower bound on the learning rate.<br>  `mode` _(defaults to "min")_: One of ``'min'``, ``'max'``, or ``'auto'``, which defines the direction of optimization, so as to adjust the learning rate <br> accordingly, i.e when a certain metric ceases improving.<br> `factor` _(defaults to 0.0)_: Factor by which the learning rate will be reduced: ``new_lr = lr * factor``.<br> `threshold` _(defaults to 1e-4)_: Threshold for measuring the new optimum, to only focus on significant changes.<br> `verbose` _(defaults to False)_: If ``True``, prints a message to ``stdout`` for each update.<br> `step_when` _(defaults to "val")_: The step type at which the scheduler should be invoked: ``'train'``, ``'val'``, or ``'epoch'``.<br> |
| step           | `step_size` _(defaults to 1)_: Period of learning rate decay, by default 1.<br> `gamma` _(defaults to 0.99)_: Multiplicative factor of learning rate decay, by default 0.99.<br> `step_when` _(defaults to "epoch")_: The step type at which the scheduler should be invoked: ``'train'``, ``'val'``, or ``'epoch'``.<br>                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |

The following code snippet describes how to initialize these objects:

```python
from ablator import OptimizerConfig, SchedulerConfig

optimizer_config = OptimizerConfig(name="sgd", arguments={"lr": 0.1})
scheduler_config = SchedulerConfig(name="cycle", arguments={"max_lr": 0.5, "total_steps": 50})
```

### Running Configurations

Running configurations defines the environment of an experiment (experiment main directory, number of checkpoints to maintain, hardware device to use, etc.). There are 2 types of running configurations:

- `RunConfig` for prototype experiments
- `ParallelConfig` for ablation experiments

#### `RunConfig` for prototype experiments

The table below summarizes the parameters:

| Parameter           | Usage                                                                                                                                                                                                                                                                                                             |
|---------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| experiment_dir _(Stateless[Optional[str]], defaults to None)_      | location to store experiment artifacts.                                                                                                                                                                                                                                                                           |
| random_seed _(Optional[int], defaults to None)_         | random seed.                                                                                                                                                                                                                                                                                                      |
| train_config _(TrainConfig)_        | training configuration. (check ``TrainConfig`` for more details)                                                                                                                                                                                                                                                  |
| model_config _(ModelConfig)_        | model configuration. (check ``ModelConfig`` for more details)                                                                                                                                                                                                                                                     |
| keep_n_checkpoints _(Stateless[int], defaults to 3)_  | number of latest checkpoints to keep.                                                                                                                                                                                                                                                                             |
| tensorboard _(Stateless[bool], defaults to True)_         | whether to use tensorboardLogger.                                                                                                                                                                                                                                                                                 |
| amp _(Stateless[bool], defaults to True)_                 | whether to use automatic mixed precision when running on gpu.                                                                                                                                                                                                                                                     |
| device _(Stateless[str], defaults to "cuda")_              | device to run on.                                                                                                                                                                                                                                                                                                 |
| verbose _(Stateless[Literal["console", "progress", "silent"]], defaults to "console")_             | verbosity level.                                                                                                                                                                                                                                                                                                  |
| eval_subsample _(Stateless[float], defaults to 1)_      | fraction of the dataset to use for evaluation.                                                                                                                                                                                                                                                                    |
| metrics_n_batches _(Stateless[int], defaults to 32)_   | max number of batches stored in every tag(train, eval, test) for evaluation.                                                                                                                                                                                                                                      |
| metrics_mb_limit _(Stateless[int], defaults to 100)_    | max number of megabytes stored in every tag(train, eval, test) for evaluation.                                                                                                                                                                                                                                    |
| early_stopping_iter _(Stateless[Optional[int]], defaults to None)_ | The maximum allowed difference between the current iteration and the last <br />iteration with the best metric before applying early stopping. Early stopping <br />will be triggered if the difference ``(current_itr-best_itr)`` exceeds ``early_stopping_iter``.<br />If set to ``None``, early stopping will not be applied. |
| eval_epoch _(Stateless[float], defaults to 1)_          | The epoch interval between two evaluations.                                                                                                                                                                                                                                                                       |
| log_epoch _(Stateless[float], defaults to 1)_           | The epoch interval between two logging.                                                                                                                                                                                                                                                                           |
| init_chkpt _(Stateless[Optional[str]], defaults to None)_          | path to a checkpoint to initialize the model with.                                                                                                                                                                                                                                                                |
| warm_up_epochs _(Stateless[float], defaults to 1)_      | number of epochs marked as warm up epochs.                                                                                                                                                                                                                                                                        |
| divergence_factor _(Stateless[Optional[float]], defaults to 100)_   | if ``cur_loss > best_loss > divergence_factor``, the model is considered <br />to have diverged.                                                                                                                                                                                                                        |

#### `ParallelConfig` for ablation experiments

`ParallelConfig` is a subclass of `RunConfig`. It introduces additional attributes to configure parallel training with horizontal scaling of a single experiment (number of trials, maximum number of trials to run concurrently, the target metrics to optimize, and more).

| Parameters                                   | Usage                                                                                                                                               |
|----------------------------------------------|-----------------------------------------------------------------------------------------------------------------------------------------------------|
| total_trials: _(Optional[int])_                  | total number of trials.                                                                                                                             |
| concurrent_trials: _(int)_                       | number of trials to run concurrently.                                                                                                               |
| search_space: _(Dict[SearchSpace])_              | search space for hyperparameter search, eg. ``{"train_config.optimizer_config.arguments.lr": SearchSpace(value_range=[0, 10], value_type="int"),}`` |
| optim_metrics: _(Optional[Dict[Optim]])_         | metrics to optimize, eg. ``{"val_loss": "min"}``                                                                                                    |
| gpu_mb_per_experiment: _(int)_                   | CUDA memory requirement per experimental trial in MB. e.g. a value of 100 is equivalent to 100MB                                                    |
| search_algo: _(SearchAlgo, defaults to SearchAlgo.tpe)_     | type of search algorithm.                                                                                                                           |
| ignore_invalid_params: _(bool, defaults to False)_          | whether to ignore invalid parameters when sampling or raise an error.                                                                               |
| remote_config: _(Optional[RemoteConfig], defaults to None)_ | remote storage configuration.                                                                                                                       |

`search_space` is used to define a set of continuous or categorical/discrete values for a certain hyperparameter that you want to optimize. Refer to [Search Space basics](./Search-space-tutorial.ipynb) to learn more about how to use it.

### Configure your experiments

Now let's combine everything to configure your experiment!

We have created model config, optimizer config, and scheduler config objects in the previous sections. Now we will create `train_config` (to set up the dataset, batch size, epochs) and reference the optimizer and scheduler config objects. 

```python
from ablator import TrainConfig

train_config = TrainConfig(
    dataset="test",
    batch_size=128,
    epochs=2,
    optimizer_config=optimizer_config,
    scheduler_config=scheduler_config,
)
```

The last step is to create a `run_config` object. This object combines the `train_config` and `model_config` objects, along with runtime settings like verbosity and device. However, we also need to redefine the run config class to update its `model_config` class from `ModelConfig` to `MyModelConfig`:

```python
@configclass
class CustomRunConfig(RunConfig):
    model_config: MyModelConfig

run_config = CustomRunConfig(
    train_config=train_config,
    model_config=MyModelConfig(),
    verbose="silent",
    device="cpu",
)
```

With the configuration created, we are half-way to launching an ablation experiment. Refer to [Prototyping models](./Prototyping-models.ipynb) and [Hyperparameter Optimization](./HPO-tutorial.ipynb) tutorials for the next steps after configuration to launch the experiment.

## Ablator custom data types

An important part of Ablator's configuration system is the incorporation of custom data types, which are used to define data type for configuration attributes. The framework created three special types: **Stateless**, **Stateful**, and **Derived**. These are custom Python annotations to define configuration attributes to which the experiment state is agnostic (does not have any impact on the experiment state).

- **Stateless** attributes can take different values between trials or experiments. For example, learning rate should be stateless, as we can train models with different learning rates. Note that if you're declaring a variable to be Stateless, it must be assigned an initial value before launching the experiment.

- **Stateful** attributes, opposite to **Stateless**, must have the same value between different experiments. For example, a binary classification model should always have output size of 2. Stateful variables, defined as a primitive datatype (no annotating needed), must be assigned with values before launching the experiment.

- **Derived** attributes are **Stateful** and are un-decided at the start of the experiment. Their values are determined by internal experiment processes that can depend on other experimental attributes, e.g model input size that depends on the dataset.


<div class="alert alert-info">

Note

- The reason for creating these annotations is that Ablator supports stateful experiment design, so the configuration should be unambiguous at the initialization state. And the use of these annotations assures the unambiguity of the configuration.
- For more information about our stateful experiment design, see our paper: [ABLATOR: Robust Horizontal-Scaling of Machine Learning Ablation Experiments](https://iordanis.me/data/ablator.pdf)

</div>

## Alternatives to constructing configuration objects

There are three methods to configure an experiment: named arguments, file-based, or dictionary-based. All previous code snippets are examples of the named-arguments method. Now let's look at how file based method and dictionary based method work.

### File-based

File based configuration is a way for you to create simple configuration files. You can use `<Run config class>.load(path/to/yaml/file)` method to create configuration with values provided in the config file.

To write these config files, simply follow `yaml` syntax. Make sure that the attributes match with those in the config classes (either default config classes from ablator, or custom ones like `MyModelConfig`). The following example shows what a config yaml file looks like. We will name it `config.yaml`:

```yaml
experiment_dir: "/tmp/dir"
train_config:
  dataset: test
  batch_size: 128
  epochs: 2
  optimizer_config:
    name: sgd
    arguments:
      lr: 0.1
  scheduler_config:
    name: cycle
    arguments:
      max_lr: 0.5
      total_steps: 50
model_config:
  inp_size: 50
  hidden_dim: 100
  activation: "relu"
  dropout: 0.15
verbose: "silent"
device: "cpu"
```

Now in your code, load these values to create the config object:

```python
config = CustomRunConfig.load("path/to/yaml/file")
```

Note that since we created a custom running configuration class `CustomRunConfig` that is tied to the custom model config in the previous sections, we used `CustomRunConfig.load("path/to/yaml/file")` to load configuration from file. Otherwise, if you're not creating any subclasses, simply run `RunConfig.load("path")` or `ParallelConfig.load("path")`.

### Dictionary based

Another alternative is similar to the file-based method, but it's defining configurations in a dictionary instead of a yaml file, and then the dictionary will be passed (as keyword arguments) to the running configuration at initialization

```python
configuration = {
    "experiment_dir": "/tmp/dir",
    "train_config": {
        "dataset": "test",
        "batch_size": 128,
        "epochs": 2,
        "optimizer_config":{
            "name": "sgd",
            "arguments": {
                "lr": 0.1
            }
        },
        "scheduler_config":{
            "name": "cycle",
            "arguments":{
                "max_lr": 0.5,
                "total_steps": 50
            }
        }
    },
    "model_config": {
        "inp_size": 50,
        "hidden_dim": 100,
        "activation": "relu",
        "dropout": 0.15
    },
    "verbose": "silent",
    "device": "cpu"
}

config = CustomRunConfig(
    **configuration
)
```

## Conclusion

Now that you know how to configure experiments, you can start creating your own prototype. In the next chapter, we will learn how to write a prototype model, define necessary configurations, and launch the experiment. 